# 匯出數據

In [ ]:
import pandas as pd
from datetime import datetime

def query_products(
    start_date=None, 
    end_date=None, 
    limit=None, 
    offset=None,
    # 如：'output.csv' 或 'output.xlsx'
    export_path=None
):
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4',
        # 回傳 dict 格式
        cursorclass=pymysql.cursors.DictCursor
    )

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT id, search_keyword, title, price, unit_price, full_text, timestamp FROM coupang_products WHERE 1=1"
            params = []

            # 日期篩選
            if start_date and end_date:
                sql += " AND timestamp BETWEEN %s AND %s"
                params.extend([start_date, end_date])

            # 排序
            sql += " ORDER BY id DESC"

            # 分頁功能
            if limit:
                sql += " LIMIT %s"
                params.append(limit)
                if offset:
                    sql += " OFFSET %s"
                    params.append(offset)

            cursor.execute(sql, params)
            rows = cursor.fetchall()

            # 顯示資料
            print("查詢結果：")
            for row in rows:
                print(f"ID: {row['id']}")
                print(f"關鍵字: {row['search_keyword']}")
                print(f"標題: {row['title']}")
                print(f"價格: {row['price']}")
                print(f"每單位: {row['unit_price']}")
                print(f"完整內容: {row['full_text']}")
                print(f"時間戳記: {row['timestamp']}")
                print("-" * 80)

            # 匯出功能
            if export_path:
                df = pd.DataFrame(rows)
                if export_path.endswith(".csv"):
                    df.to_csv(
                        export_path,
                        index=False,
                        encoding="utf-8-sig"
                    )
                    print(f"資料已匯出為 CSV：{export_path}")
                elif export_path.endswith(".xlsx"):
                    df.to_excel(export_path, index=False)
                    print(f"資料已匯出為 Excel：{export_path}")
                else:
                    print("不支援的檔案格式，只支援 .csv 與 .xlsx")

if __name__ == "__main__":
    # 篩選日期（可調整）
    start_date = "2024-12-01"
    end_date = "2025-12-31"

    # 分頁參數（可調整）
    limit = 10
    offset = 0

    # 匯出路徑（None 表示不匯出）
    # 或 coupang_output.csv
    export_path = "coupang_output.xlsx"

    # 執行查詢
    query_products(
        start_date=start_date,
        end_date=end_date,
        limit=limit,
        offset=offset,
        export_path=export_path
    )
